# Final Project: Restaurant Recommander System

## Imports

In [214]:
import pandas as pd
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import f1_score

In [3]:
config_file = open("config.yml", "r")
config = yaml.load(config_file, Loader=yaml.FullLoader)
DATA_DIRECTORY = config["DATA_DIRECTORY"]

## Chargement des données

In [4]:
# Données relatives aux commandes des customers aux vendors
order_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "orders.csv"))

# Données des vendeurs(localisation, identifiant)
vendors_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "vendors.csv"))

# Données des clients (date de naissance, ID, genre)
train_customer_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "train_customers.csv"))

# Localisations des clients
train_locations_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "train_locations.csv"))

# Ensemble de tests sur les données des clients
test_customer_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "test_customers.csv"))

# Ensemble de tests sur les localisations des clients
test_locations_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "test_locations.csv"))

/run/user/22104/ipykernel_4349/3856822910.py:2: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  order_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "orders.csv"))


## Definition d'une super-classe de modèle

In [5]:
class RecommanderModel:
    model_type = "undefined"
    # Définition du schéma de sortie du modèle
    output_schema = config["output_schema"]

    def __init__(self, model_name: str) -> None:
        self.model_name = model_name
        pass

    def train(self, **kwargs) -> None:
        pass

    def predict(self, **kwargs) -> None:
        pass

    def save_predictions(self, **kwargs) -> None:
        pass

    def print_model(self):
        print(f"{self.model_type} recommander model\nmodel name: '{self.model_name}'")

In [114]:
order_with_loc_df = pd.merge(
    order_df[["customer_id", "LOCATION_NUMBER", "vendor_id", "vendor_rating","is_rated"]],
    train_locations_df,
    left_on=["customer_id", "LOCATION_NUMBER"],
    right_on=["customer_id", "location_number"],
)
order_with_loc_df["is_rated"] = order_with_loc_df["is_rated"].apply(lambda b: 1 if b =='Yes' else 0 )
order_with_loc_df["is_rated"] = order_with_loc_df["is_rated"].fillna(0)
order_with_loc_df

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude
0,92PEE24,0,105,NaN,0,0,NaN,-0.132100,-78.575297
1,92PEE24,0,105,NaN,0,0,NaN,-0.132100,-78.575297
2,92PEE24,0,105,NaN,0,0,NaN,-0.132100,-78.575297
3,QS68UD8,0,294,NaN,0,0,Work,-0.393396,-78.544417
4,MB7VY5F,0,83,NaN,0,0,NaN,-1.072823,-78.464121
...,...,...,...,...,...,...,...,...,...
135298,3S6VG6R,1,199,5.0,1,1,NaN,2.284875,0.717124
135299,ND4PIJL,0,907,NaN,0,0,NaN,13.380083,-1.387421
135300,1NRK5HF,0,105,NaN,0,0,NaN,-0.772600,0.231851
135301,QDXLWM7,1,28,NaN,0,1,NaN,1.751487,0.375234


In [157]:
train_order_df, test_order_df = train_test_split(
    order_with_loc_df, test_size=0.2, random_state=42
)
train_order_df, val_order_df = train_test_split(
    train_order_df, test_size=0.2, random_state=42
)
train_order_df

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude
121366,5ARTXD7,1,681,5.0,1,1,NaN,-0.036682,-0.069342
63263,JTDL6KB,0,386,0.0,0,0,NaN,-0.492942,0.179312
30695,H9REAWE,0,78,NaN,0,0,Home,-0.025458,-78.587900
80592,7I7SY2D,1,4,NaN,0,1,NaN,0.415142,0.931376
132382,EIA3K80,3,92,NaN,0,3,Other,0.107888,0.596460
...,...,...,...,...,...,...,...,...,...
25191,4QNVW0N,1,176,NaN,0,1,Other,0.141303,-78.607609
35276,NBHJZZB,0,78,NaN,0,0,NaN,0.275762,-78.623499
124517,3I2NFF0,0,289,NaN,0,0,NaN,0.431060,-0.005375
51877,0Z52KAY,1,843,NaN,0,1,NaN,-0.795356,0.082256


In [161]:
train_order_df[(train_order_df['vendor_rating'] ==0) & (train_order_df['is_rated'] == np.nan) ]

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude


In [166]:
train_order_df[(~ np.isnan(train_order_df['vendor_rating'])) ]

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude
121366,5ARTXD7,1,681,5.0,1,1,NaN,-0.036682,-0.069342
63263,JTDL6KB,0,386,0.0,0,0,NaN,-0.492942,0.179312
83059,YW8TF5B,1,207,0.0,0,1,NaN,0.062256,-0.004625
42761,LBD9UF5,1,83,0.0,0,1,NaN,0.453676,0.011345
53418,7D0W4LG,1,83,0.0,0,1,NaN,0.158469,0.045799
...,...,...,...,...,...,...,...,...,...
46830,W7OB6BY,1,90,0.0,0,1,NaN,-0.289262,0.140932
80629,YI89Y35,2,113,0.0,0,2,NaN,0.128789,0.450035
46688,8DFPL4M,1,78,0.0,0,1,NaN,0.598227,0.559265
69902,DIQ1RUF,1,75,0.0,0,1,NaN,0.147127,0.539187


In [147]:



# Créer une table pivot en utilisant 'customer_id', 'vendor_id' et 'LOCATION_NUMBER' comme index
Y_matrix = pd.pivot_table(train_order_df, 
                                values='vendor_rating', 
                                index='customer_id', 
                                columns='vendor_id', 
                                fill_value=0)

# Afficher la matrice d'évaluation utilisateur-vendeur
Y_matrix

vendor_id,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
customer_id,,,,,,,,,,,,,,,,,,,,,
000THBA,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00GV4J4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00HWUU3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00OT8JX,2.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
00QIIMH,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZP5BHU,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZRJABJ,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZVGIVK,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
np.count_nonzero(Y_matrix)
Y_matrix.values.shape

(11362, 100)

In [151]:
rating = pd.pivot_table(train_order_df, values='is_rated', index='customer_id', 
                                columns='vendor_id',
                                fill_value=0)

rating

vendor_id,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
customer_id,,,,,,,,,,,,,,,,,,,,,
000THBA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005ECL6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0075AM7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009UFS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00F8I3F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZVGIVK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZVLIB5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZWKMGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
n_ratings = np.sum(rating.values)
n_ratings

In [29]:

# Effectuer la décomposition en valeurs singulières
matrix = Y_matrix.values
u, s, vh = np.linalg.svd(matrix, full_matrices=False)

In [ ]:
def cosine_similarity(v: np.ndarray, u: np.ndarray) -> float:
    return (np.dot(v, u)) / (np.linalg.norm(v) * np.linalg.norm(u))


## Funk SVD

### Initialize Matrices

In [100]:
latent_dim = 100
seed = 16
n_customer = 50
n_vendor = 50
np.random.seed(seed)
customer_mat = np.random.rand(n_customer, latent_dim)
vendor_mat = np.random.rand(latent_dim,n_vendor)
np.dot(customer_mat,vendor_mat).shape

(50, 50)

In [239]:
VAL_KEY = 'val'
PRED_KEY = 'pred'
VALIDATION_KEY = 'validation'
TRAIN_KEY = 'training'
num_train_rows = 50
num_val_rows = 5
# Générer des valeurs aléatoires pour les colonnes 'val' et 'pred' pour chaque dataframe
train_set = pd.DataFrame({
    VAL_KEY: np.random.randint(low=1, high=10, size=num_train_rows),
    PRED_KEY: np.random.randint(low=0, high=2, size=num_train_rows)
})

val_set = pd.DataFrame({
    VAL_KEY: np.random.randint(low=1, high=10, size=num_val_rows),
    PRED_KEY: np.random.randint(low=0, high=2, size=num_val_rows)
})


In [241]:
def generate_comparison_df(train_set,val_set):
    train_set = train_set.rename(columns={PRED_KEY: TRAIN_KEY })
    val_set = val_set.rename(columns={PRED_KEY: VALIDATION_KEY})
    comparison_df = pd.merge(train_set,val_set,on=VAL_KEY)
    return comparison_df

def calculate_f1_score(train_set,val_set):
    df = generate_comparison_df(train_set,val_set)
    true_labels = df[VALIDATION_KEY]
    predicted_labels = df[TRAIN_KEY]

    f1 = f1_score(true_labels, predicted_labels)

    return f1

calculate_f1_score(train_set,val_set)

0.6060606060606061